In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import csv

c:\Users\cherr\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = './Sentiment_classification_model'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.eval()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\cherr\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\cherr\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\cherr\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\cherr\AppData\Roaming\Python\Python310\site-packages\traitlets\config\applicatio

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [3]:
original_text = 'original_text.csv'
modified_text = 'output.csv'

In [4]:
def verify_similarity(original, modified, model, tokenizer):
    # 确保模型处于评估模式
    #model.eval()
    model = model.distilbert #访问底层模型

    # 对原始文本和修改后的文本进行编码
    original_encoding = tokenizer(original, return_tensors='pt', padding=True, truncation=True, max_length=512)
    modified_encoding = tokenizer(modified, return_tensors='pt', padding=True, truncation=True, max_length=512)

    with torch.no_grad():
        # 获取原始文本的隐藏状态
        original_outputs = model(**original_encoding)
        original_hidden_state = original_outputs.last_hidden_state.mean(dim=1)

        # 获取修改后文本的隐藏状态
        modified_outputs = model(**modified_encoding)
        modified_hidden_state = modified_outputs.last_hidden_state.mean(dim=1)

        # 计算余弦相似度
        similarity = F.cosine_similarity(original_hidden_state, modified_hidden_state)
        return similarity.item()

In [19]:
#读取原始文本文件
import random


original_texts = []
original_labels = []
with open(original_text, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        #去掉第一行
        if row[0] == 'id':
            continue
        original_texts.append(row[1])
        original_labels.append(row[3])
#读取修改后文本文件
modified_texts = []
with open(modified_text, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        #去掉第一行
        if row[0] == 'id':
            continue
        modified_texts.append(row[1])     
success=0
#循环修改modify文本
for i in range(len(original_texts)):
    original = original_texts[i]
    #进入死循环
    x=0
    while True:
        modified = modified_texts[i]
        #从modified中随机选取三个单词，改变其中两个相邻字母的顺序
        #随机独立选取三个单词
        words = modified.split(' ')
        if len(words) < 3:
            continue
        idx = random.randint(0, len(words) - 3)
        if len(words[idx]) >= 2:
            idx1 = random.randint(0, len(words[idx]) - 2)
            idx2 = idx1 + 1
            word1 = words[idx][:idx1] + words[idx][idx2] + words[idx][idx1] + words[idx][idx2 + 1:]
        if len(words[idx + 1]) >= 2:
            idx3 = random.randint(0, len(words[idx + 1]) - 2)
            idx4 = idx3 + 1
            word2 = words[idx + 1][:idx3] + words[idx + 1][idx4] + words[idx + 1][idx3] + words[idx + 1][idx4 + 1:]
        if len(words[idx + 2]) >= 2:
            idx5 = random.randint(0, len(words[idx + 2]) - 2)
            idx6 = idx5 + 1
            word3 = words[idx + 2][:idx5] + words[idx + 2][idx6] + words[idx + 2][idx5] + words[idx + 2][idx6 + 1:]
        words[idx] = word1
        words[idx + 1] = word2
        words[idx + 2] = word3
        modified = ' '.join(words)
        
        
        
        
        
        
        
        similarity = verify_similarity(original, modified, model, tokenizer)
        input = tokenizer(modified, return_tensors='pt', padding=True, truncation=True, max_length=512)
        output = model(**input)
        pt_predection = F.softmax(output[0], dim=-1)
        #解析pt_predection的结果
        pt_predection = pt_predection.tolist()
        #判断pt_predection的结果落在哪一项
        pt_predection = pt_predection[0].index(max(pt_predection[0]))
        
        print(f'original: {original}')
        print(f'modified: {modified}')
        print(f'similarity: {similarity}')
        print(f'pt_predection: {pt_predection}')
        print(f'original_labels: {original_labels[i]}')
        print(f'id: {i}')
        print(f'success: {success}')
        #如果相似度大于0.75并且预测结果和原始标签不一致，修改成功
        if similarity > 0.75 and pt_predection != int(original_labels[i]):
            success=success+1
            #修改写回modified_texts
            modified_texts[i] = modified
            break
        
        

        
        
        #超过100次不成功就退出
        x=x+1
        if x>100:
            break
print('success num:',success)
#将修改后的文本写入时间区分的新文件
with open('output.csv', 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'attacked_text'])
    for i, text in enumerate(modified_texts):
        writer.writerow([i, text])



original:   #powerblog What is this powerblog challenge you keep talking about?  I`m a newbie follower
modified:   #powerblog What is this powerblog challenge you keep tlaking abuot? ni I`m a newbie follower
similarity: 0.9208240509033203
pt_predection: 0
original_labels: 1
id: 0
success: 0
original: Good mornin. Today will end early, woo. Gonna work on rick`s surprise PROJECT DUE ON TUESDAY
modified: Good mornin. Today will end early, woo. Gonna work on rick`s surprise PROJCET UDE NO TUESDAY
similarity: 0.9388962984085083
pt_predection: 2
original_labels: 2
id: 1
success: 1
original: Good mornin. Today will end early, woo. Gonna work on rick`s surprise PROJECT DUE ON TUESDAY
modified: Good mornin. Today will end early, woo. Gonna work on rick`s surprise PRJOECT UDE NO TUESDAY
similarity: 0.9467256665229797
pt_predection: 2
original_labels: 2
id: 1
success: 1
original: Good mornin. Today will end early, woo. Gonna work on rick`s surprise PROJECT DUE ON TUESDAY
modified: Good mornin. To

In [ ]:
ori_test =  '"Good mornin. Today will end early, woo. Gonna work on rick`s surprise PROJECT DUE ON TUESDAY"'
test_text = '"Godo omnrin. Today will end early, woo. Gonna work on rcik`s surrpise PROEJCT DUE ON TUEDSAY"'
print(verify_similarity(ori_test, test_text, model, tokenizer))
input = tokenizer(test_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
output = model(**input)
pt_predection = F.softmax(output[0], dim=-1)
print(pt_predection)

0.6760171055793762
tensor([[0.2363, 0.5577, 0.2059]], grad_fn=<SoftmaxBackward0>)


In [ ]:
model.distilbert(**input).last_hidden_state.mean()

tensor(-0.0141, grad_fn=<MeanBackward0>)